## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/weather_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,54.32,79,76,5.48,light rain
1,1,Viedma,AR,-40.8135,-62.9967,68.05,36,100,4.81,overcast clouds
2,2,Puerto Ayora,EC,-0.7393,-90.3518,81.55,88,94,7.00,overcast clouds
3,3,Turan,VN,16.0678,108.2208,75.18,88,20,1.14,few clouds
4,4,Torbay,CA,47.6666,-52.7314,49.28,88,100,2.30,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? -10
What is the maximum temperature you would like for your trip? 120


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,54.32,79,76,5.48,light rain
1,1,Viedma,AR,-40.8135,-62.9967,68.05,36,100,4.81,overcast clouds
2,2,Puerto Ayora,EC,-0.7393,-90.3518,81.55,88,94,7.00,overcast clouds
3,3,Turan,VN,16.0678,108.2208,75.18,88,20,1.14,few clouds
4,4,Torbay,CA,47.6666,-52.7314,49.28,88,100,2.30,overcast clouds
5,5,Praia Da Vitoria,PT,38.7333,-27.0667,61.05,59,40,29.33,scattered clouds
6,6,Goderich,CA,43.7501,-81.7165,37.90,75,87,10.36,overcast clouds
7,7,Yulara,AU,-25.2406,130.9889,60.64,29,0,1.14,clear sky
8,8,Port Elizabeth,ZA,-33.9180,25.5701,67.10,95,40,9.22,scattered clouds
9,9,Mahebourg,MU,-20.4081,57.7000,77.29,83,75,10.36,broken clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 667 entries, 0 to 677
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              667 non-null    int64  
 1   City                 667 non-null    object 
 2   Country              662 non-null    object 
 3   Lat                  667 non-null    float64
 4   Lng                  667 non-null    float64
 5   Max Temp             667 non-null    float64
 6   Humidity             667 non-null    int64  
 7   Cloudiness           667 non-null    int64  
 8   Wind Speed           667 non-null    float64
 9   Current Description  667 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 57.3+ KB


In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 662 entries, 0 to 677
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              662 non-null    int64  
 1   City                 662 non-null    object 
 2   Country              662 non-null    object 
 3   Lat                  662 non-null    float64
 4   Lng                  662 non-null    float64
 5   Max Temp             662 non-null    float64
 6   Humidity             662 non-null    int64  
 7   Cloudiness           662 non-null    int64  
 8   Wind Speed           662 non-null    float64
 9   Current Description  662 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 56.9+ KB


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,54.32,light rain,-33.6500,115.3333,
1,Viedma,AR,68.05,overcast clouds,-40.8135,-62.9967,
2,Puerto Ayora,EC,81.55,overcast clouds,-0.7393,-90.3518,
3,Turan,VN,75.18,few clouds,16.0678,108.2208,
4,Torbay,CA,49.28,overcast clouds,47.6666,-52.7314,
5,Praia Da Vitoria,PT,61.05,scattered clouds,38.7333,-27.0667,
6,Goderich,CA,37.90,overcast clouds,43.7501,-81.7165,
7,Yulara,AU,60.64,clear sky,-25.2406,130.9889,
8,Port Elizabeth,ZA,67.10,scattered clouds,-33.9180,25.5701,
9,Mahebourg,MU,77.29,broken clouds,-20.4081,57.7000,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,54.32,light rain,-33.6500,115.3333,Observatory Guest House
1,Viedma,AR,68.05,overcast clouds,-40.8135,-62.9967,Hotel Cristal
2,Puerto Ayora,EC,81.55,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Turan,VN,75.18,few clouds,16.0678,108.2208,Grand Mercure Danang
4,Torbay,CA,49.28,overcast clouds,47.6666,-52.7314,Gallows Cove
...,...,...,...,...,...,...,...
673,Marsa Matruh,EG,62.80,few clouds,31.3525,27.2453,Beau Site
674,Kanniyakumari,IN,82.49,overcast clouds,8.0833,77.5667,Sparsa Kanyakumari
675,Kamuli,UG,71.44,overcast clouds,0.9472,33.1197,Century Hotel Limited And laundry Services
676,Nikel,RU,35.78,overcast clouds,69.4128,30.2220,Severnoe Siyanie


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 597 entries, 0 to 677
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 597 non-null    object 
 1   Country              597 non-null    object 
 2   Max Temp             597 non-null    float64
 3   Current Description  597 non-null    object 
 4   Lat                  597 non-null    float64
 5   Lng                  597 non-null    float64
 6   Hotel Name           597 non-null    object 
dtypes: float64(3), object(4)
memory usage: 37.3+ KB


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))